<a href="https://colab.research.google.com/github/DeenReinierCrusem/CPEN65-1/blob/main/Laboratory_Exercise_1_Intro_to_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Laboratory Exercise 1: Intro to Machine Learning

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = [
 {'kernel': ['linear'], 'C': [10., 30., 100., 300., 1000., 3000., 10000., 30000.0]},
 {'kernel': ['rbf'], 'C': [1.0, 3.0, 10., 30., 100., 300., 1000.0],
 'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]},
]
svm_reg = SVR()
grid_search = GridSearchCV(svm_reg, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(housing_prepared, housing_labels)
GridSearchCV(cv=5, estimator=SVR(),
 param_grid=[{'C': [10.0, 30.0, 100.0, 300.0, 1000.0, 3000.0,
 10000.0, 30000.0],
 'kernel': ['linear']},
 {'C': [1.0, 3.0, 10.0, 30.0, 100.0, 300.0, 1000.0],
 'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0],
 'kernel': ['rbf']}],
 scoring='neg_mean_squared_error', verbose=2)
negative_mse = grid_search.best_score_
rmse = np.sqrt(-negative_mse)
rmse
70363.84006944533
grid_search.best_params_
{'C': 30000.0, 'kernel': 'linear'}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon, reciprocal
# see https://docs.scipy.org/doc/scipy/reference/stats.html
# for `expon()` and `reciprocal()` documentation and more probability distribution functions.
# Note: gamma is ignored when kernel is "linear"
param_distribs = {
 'kernel': ['linear', 'rbf'],
 'C': reciprocal(20, 200000),
 'gamma': expon(scale=1.0),
}
svm_reg = SVR()
rnd_search = RandomizedSearchCV(svm_reg, param_distributions=param_distribs,
 n_iter=50, cv=5, scoring='neg_mean_squared_error',
 verbose=2, random_state=42)
rnd_search.fit(housing_prepared, housing_labels)
RandomizedSearchCV(cv=5, estimator=SVR(), n_iter=50,
 param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f9bd002c790>,
 'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f9bd002cb10>,
 'kernel': ['linear', 'rbf']},
 random_state=42, scoring='neg_mean_squared_error',
 verbose=2)
negative_mse = grid_search.best_score_
rmse = np.sqrt(-negative_mse)
rmse
54767.960710084146
rnd_search.best_params_
{'C': 157055.10989448498, 'gamma': 0.26497040005002437, 'kernel': 'rbf'}
expon_distrib = expon(scale=1.)
samples = expon_distrib.rvs(10000, random_state=42)
plt.figure(figsize=(10, 4))
plt.subplot(121)
plt.title("Exponential distribution (scale=1.0)")
plt.hist(samples, bins=50)
plt.subplot(122)
plt.title("Log of this distribution")
plt.hist(np.log(samples), bins=50)
plt.show()

reciprocal_distrib = reciprocal(20, 200000)
samples = expon_distrib.rvs(10000, random_state=42)
plt.figure(figsize=(10, 4))
plt.subplot(121)
plt.title("Exponential distribution (scale=1.0)")
plt.hist(samples, bins=50)
plt.subplot(122)
plt.title("Log of this distribution")
plt.hist(np.log(samples), bins=50)
plt.show()

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
def indices_of_top_k(arr, k):
return np.sort(np.argpartition(np.array(arr), -k)[-k:])
class TopFeatureSelector(BaseEstimator, TransformerMixin):
 def __init__(self, feature_importances, k):
 self.feature_importances = feature_importances
 self.k = k
 def fit(self, X, y=None):
 self.feature_indices_ = indices_of_top_k(self.feature_importances, self.k)
 return self
 def transform(self, X):
 return X[:, self.feature_indices_]
k = 5
top_k_feature_indices = indices_of_top_k(feature_importances, k)
top_k_feature_indices
array([ 0, 1, 7, 9, 12])
np.array(attributes)[top_k_feature_indices]
array(['longitude', 'latitude', 'median_income', 'pop_per_hhold',
 'INLAND'], dtype='<U18')
sorted(zip(feature_importances, attributes), reverse=True)[:k]
[(0.36615898061813423, 'median_income'),
 (0.16478099356159054, 'INLAND'),
 (0.10879295677551575, 'pop_per_hhold'),
 (0.07334423551601243, 'longitude'),
 (0.06290907048262032, 'latitude')]
[4] Try creating a single pipeline that does the full data preparation plus the final
prediction.
[5] Automatically explore some preparation options using GridSearchCV.
preparation_and_feature_selection_pipeline = Pipeline([
 ('preparation', full_pipeline),
 ('feature_selection', TopFeatureSelector(feature_importances, k))
])
housing_prepared_top_k_features = preparation_and_feature_selection_pipeline.fit_transform(housing)
housing_prepared_top_k_features[0:3]
array([[-1.15604281, 0.77194962, -0.61493744, -0.08649871, 0. ],
 [-1.17602483, 0.6596948 , 1.33645936, -0.03353391, 0. ],
 [ 1.18684903, -1.34218285, -0.5320456 , -0.09240499, 0. ]])
housing_prepared[0:3, top_k_feature_indices]
array([[-1.15604281, 0.77194962, -0.61493744, -0.08649871, 0. ],
 [-1.17602483, 0.6596948 , 1.33645936, -0.03353391, 0. ],
 [ 1.18684903, -1.34218285, -0.5320456 , -0.09240499, 0. ]])

In [ ]:
prepare_select_and_predict_pipeline = Pipeline([
 ('preparation', full_pipeline),
 ('feature_selection', TopFeatureSelector(feature_importances, k)),
('svm_reg', SVR(**rnd_search.best_params_))
])
prepare_select_and_predict_pipeline.fit(housing, housing_labels)
Pipeline(steps=[('preparation',
 ColumnTransformer(transformers=[('num',
 Pipeline(steps=[('imputer',
 SimpleImputer(strategy='median')),
 ('attribs_adder',
 CombinedAttributesAdder()),
 ('std_scaler',
 StandardScaler())]),
 ['longitude', 'latitude',
'housing_median_age',
'total_rooms',
 'total_bedrooms',
 'population', 'households',
'median_income']),
 ('cat', OneHotEncoder(...
 TopFeatureSelector(feature_importances=array([7.33442355e-02, 6.29090705e-02, 4.11437985e-02,
1.46726854e-02,
 1.41064835e-02, 1.48742809e-02, 1.42575993e-02, 3.66158981e-01,
 5.64191792e-02, 1.08792957e-01, 5.33510773e-02, 1.03114883e-02,
 1.64780994e-01, 6.02803867e-05, 1.96041560e-03, 2.85647464e-03]),
 k=5)),
 ('svm_reg',
 SVR(C=157055.10989448498, gamma=0.26497040005002437))])
some_data = housing.iloc[:4]
some_labels = housing_labels.iloc[:4]
print("Predictions:\t", prepare_select_and_predict_pipeline.predict(some_data))
print("Labels:\t\t", list(some_labels))
Predictions: [203214.28978849 371846.88152572 173295.65441612 47328.3970888 ]
Labels: [286600.0, 340600.0, 196900.0, 46300.0]

In [ ]:
full_pipeline.named_transformers_["cat"].handle_unknown = 'ignore'
param_grid = [{
 'preparation__num__imputer__strategy': ['mean', 'median', 'most_frequent'],
'feature_selection__k': list(range(1, len(feature_importances) + 1))
}]
grid_search_prep = GridSearchCV(prepare_select_and_predict_pipeline, param_grid, cv=5,
 scoring='neg_mean_squared_error', verbose=2)
grid_search_prep.fit(housing, housing_labels)
GridSearchCV(cv=5,
 estimator=Pipeline(steps=[('preparation',
 ColumnTransformer(transformers=[('num',
 Pipeline(steps=[('imputer',
 SimpleImputer(strategy='median')),
 ('attribs_adder', CombinedAttributesAdder()),
('std_scaler', StandardScaler())]),
 ['longitude',
 'latitude',
'housing_median_age',
'total_rooms',
'total_bedrooms',
'population',
 'households',
 'median_inc...
 5.64191792e-02, 1.08792957e-01, 5.33510773e-02, 1.03114883e-02,
 1.64780994e-01, 6.02803867e-05, 1.96041560e-03, 2.85647464e-03]),
 k=5)),
 ('svm_reg',
 SVR(C=157055.10989448498,
 gamma=0.26497040005002437))]),
 param_grid=[{'feature_selection__k': [1, 2, 3, 4, 5, 6, 7, 8, 9,
 10, 11, 12, 13, 14, 15, 16],
 'preparation__num__imputer__strategy': ['mean',
 'median',
 'most_frequent']}],
 scoring='neg_mean_squared_error', verbose=2)
grid_search_prep.best_params_
{'feature_selection__k': 15,
'preparation__num__imputer__strategy': 'most_frequent'}